## LeBronto Neighborhoods
### Andrew Rodgers
Import all the necessary modules and make scrape the data from the webpage

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

resp = requests.get(url=url)

soup = BeautifulSoup(resp.text,"html.parser")

table = soup.findAll('table')[0]

Format the html to list, one for the header and another for the data

In [4]:
header = []
data = []
for i in table.findAll('tr'):
    for j in i.findAll('th'):
        header.append(j.text)
    for j in i.findAll('td'):
        data.append(j.text)

Format the header and data lists into a *pandas* dataframe, **df**

In [5]:
header[2] = header[2][0:-1]
col1 = []
col2 = []
col3 = []
for i in range(0,len(data),3):
    col1.append(data[i])
    col2.append(data[i+1])
    col3.append(data[i+2])
    
df = pd.DataFrame(list(zip(col1,col2,col3)),columns = header)

Subset **df** so we only use the rows with assigned Boroughs  
Use the Borough for the neighborhood name if not assigned

In [6]:
df = df[df.Borough != 'Not assigned']

for i in range(0,len(df['Postcode'])):
    if "\n" in df.iloc[i,2]:
        df.iloc[i,2] = df.iloc[i,2][0:-1]
    if df.iloc[i,2] == "Not assigned":
        df.iloc[i,2] = df.iloc[i,1]

Sort **df** by Postcode to make searchign easier  
Loop through **df** to combine neighborhoods in the same cell, separated by commas

In [7]:
df = df.sort_values('Postcode',0)

for i in range(0,len(df['Postcode'])):
    if df.iloc[i-1,0] == df.iloc[i,0]:
        #if df.iloc[i-1,2] in df.iloc[i,2]:
        df.iloc[i,2] = df.iloc[i-1,2] + ", " + df.iloc[i,2]
        
df = df.drop_duplicates(subset='Postcode', keep='last', inplace=False)

Here's a sample of **df**

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
12,M1B,Scarborough,"Rouge, Malvern"
27,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
44,M1E,Scarborough,"Guildwood, Morningside, West Hill"
53,M1G,Scarborough,Woburn
62,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)

In [11]:
geo = pd.read_csv("http://cocl.us/Geospatial_data")

In [12]:
df['Latitude'] = geo['Latitude']
df['Longitude'] = geo['Longitude']

In [13]:
lat = []
long = []
for i in df['Postcode']:
    lat.append(float(geo.loc[geo['Postal Code']==i]['Latitude']))
    long.append(float(geo.loc[geo['Postal Code']==i]['Longitude']))
    
df['Latitude'] = lat
df['Longitude'] = long

In [14]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
12,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
27,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
44,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
53,M1G,Scarborough,Woburn,43.770992,-79.216917
62,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
